<a href="https://colab.research.google.com/github/vincentmartin/tp-fine-tuning-student-version/blob/main/tp-fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP fine-tuning de LLM

Dans ce notebook vous allez fine tuner un LLM de base, Flan T5, avec la technique PEFT et LoRA.

### Instruction à suivre pour exécution sur Google Colab

Aller dans `Execution -> Modifier le type d'exécution` puis sélectionner `T4-GPU` pour exploiter les fonctionnalités GPU.

![Colab GPU](resources/colab_gpu.png "T4-GPU")

Installationd des dépendances

In [ ]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    evaluate \
    rouge_score \
    loralib \
    peft

Import des dépendances

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"

Chargement du LLM de base.

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Création d'une fonction pour afficher le nombre de paramètres entraînables.

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

## Fine tuning avec PEFT et LoRA

Le fine tuning complet d'un modèle n'est pas un choix judicieux pour un particulier ou une entreprise qui n'a pas une énorme puissance de calcul. La méthode la plus appropriée est d'utiliser PEFT (_Parameter Efficient Fine-Tuning_).

PEFT est un ensemble de technique qui incluant LORA (_Low Rank Adaptation_) et le _prompt tuning_ (**différent du prompt engineering**). LORA permet de fine tuner un modèle avec peu de ressources matérielles (un ou deux GPU). LORA permet de créer des adapteurs composés de 1-10% des paramètres du LLM original. De plus, le LLM original n'est pas modifié, ce qui permet de rapidement changer d'adapteurs en fonction du cas d'usage.

### Configuration de PEFT / LoRA

Premièrement, configurons PEFT/LoRA pour fine tuner notre modèle de base avec ce que l'on appelle _adapteur_. 

PEFT/LoRA gêle les couches du LLM original pour n'entraîner que l'adapteur.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank : plus il est grand, plus il y a de paramètres. Idéal : 16-32
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # Pour FLANT5, laisser ce type
)

Ajouter l'adapteur au LLM original.

In [ ]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

## Lancement de l'entraînement

Chargeons le jeu de données pour l'entraînement.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Pour que l'entraînement prenne un temps acceptable dans ce notebook, nous diminuons la taille du jeu de données.

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

**Exercice**  : en vous aidant de la documentation https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments, créer une instance de **Trainer** pour entraîner le LLM. Vous utiliserez les paramètres suivants : 
- auto_find_batch_size=True,
- learning_rate=1e-3, 
- num_train_epochs=5,
- logging_steps=1,
- max_steps=1   

Le jeu de données à utiliser pour l'entraînement est `tokenized_datasets["train"]`.

**Dans Google Colab, utiliser `report_to=None` sinon il vous sera demandé une clef Wanadb.**

In [ ]:
output_dir = './training-output'

peft_training_args = "" # A COMPLETER
    
peft_trainer =  "" # A COMPLETER

**Exercice** : Lancer l'entraînement et sauvegarder le modèle (adapteur)  et le tokenizer dans le dossier `training-output-checkpoint`.

In [ ]:
# A COMPLETER

### Evaluation du modèle fine tuné

Une erreur classique lorsque l'on début est d'évaluer les performances en 'regardant' quelques générations manuellement. C'est une mauvaise idée car (1) ce n'est pas quantifié et (2) ce qui fonctionne sur quelques exemples ne fonctionne peut être pas sur des milliers d'exemples (principe de généralisation).

Lorsque l'on fine tune un modèle, il est donc capital de mesurer les performances pour savoir si **globalement** les résultats sont meilleurs.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       'training-output-checkpoint', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)


In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

device = "cuda" if torch.cuda.is_available() else "cpu"


prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

original_model_outputs = original_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


peft_model_outputs = peft_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'RESUME HUMAIN:\n{human_baseline_summary}')
print(dash_line)
print(f'RESUME AVEC MODELE ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(dash_line)
print(f'RESUME AVEC MODELE PEFT: {peft_model_text_output}')

Inférence sur 10 exemples du jeu de test.

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]
    
    original_model_outputs = original_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

**Exercice** : en utilisant la documentation https://huggingface.co/docs/evaluate/main/en/choosing_a_metric, calculer le score ROUGE entre : 
- Les résumés du modèle original  (`original_model_summaries`)  vs. résumés humain (`human_baseline_summaries`).
- Les résumés du modèle peft  (`peft_model_summaries`) vs. résumé humain (`human_baseline_summaries`).

Afficher les scores et commentez les.

In [ ]:
# A COMPLETER

**Exercice** : calculer le gain de performance en pourcentage du modèle PEFT sur le modèle original

In [ ]:
# A COMPLETER